# Setup

In [ ]:
import json
import logging
import os
import sys
from pathlib import Path

from ChurchToolsApi import ChurchToolsApi

parent_path = str(Path().absolute().parent)
if parent_path not in sys.path:
    print(f"adding {parent_path} to parent path")
    sys.path.insert(1, parent_path)

from main import (
    add_id_to_local_song_if_available_in_ct,
    clean_all_songs,
    download_missing_online_songs,
    get_ct_songs_as_df,
    read_songs_to_df,
    upload_new_local_songs_and_generate_ct_id,
    write_df_to_file,
)


In [ ]:
logger = logging.getLogger(__name__)

config_file = Path("../logging_config.json")
with config_file.open(encoding="utf-8") as f_in:
    logging_config = json.load(f_in)
    log_directory = Path(logging_config["handlers"]["file"]["filename"]).parent
    if not log_directory.exists():
        log_directory.mkdir(parents=True)
    logging.config.dictConfig(config=logging_config)
logger.info("update.ipynb - Jupyter Notebook")


In [ ]:
ct_domain = os.getenv("CT_DOMAIN")
ct_token = os.getenv("CT_TOKEN")

if ct_domain is None or ct_token is None:
    from secure.config import ct_domain, ct_token

    logger.info(
        "ct_domain or ct_token missing in env variables - using local config instead"
    )
    from secure import config

    ct_domain = config.ct_domain
    ct_token = config.ct_token

api = ChurchToolsApi(domain=ct_domain, ct_token=ct_token)

connected_as = api.who_am_i()

print(f"This notebook will connect to {ct_domain} as {connected_as['firstName']} {connected_as['lastName']}")

# Read local SNG files
* read songs
* clean songs
* overwrite songs

In [ ]:
songs_temp = []
df_sng = read_songs_to_df()
df_sng = clean_all_songs(df_sng=df_sng)
write_df_to_file(df_sng)

## Get CT songs 
* update local IDs
* upload new local songs
* download missing online songs

In [ ]:
# Match all SongIDs from CT to local songs where missing
df_ct = get_ct_songs_as_df(api)
add_id_to_local_song_if_available_in_ct(df_sng, df_ct)

# Upload all songs into CT that are new
df_ct = get_ct_songs_as_df(api)
upload_new_local_songs_and_generate_ct_id(df_sng, df_ct)


In [ ]:
download_missing_online_songs(df_sng, df_ct, api)

In [ ]:

"""
df_sng = read_baiersbronn_songs_to_df()
df_ct = get_ct_songs_as_df()
upload_local_songs_by_id(df_sng, df_ct)
"""
